In [1]:
import os
import math
import shutil
import random
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from multiprocessing import Pool
from collections import OrderedDict

In [2]:
i = 0
class data_preprocessing:
    
    uc = []
    splits = []
    im_class_map = {}
    train_class_map = {}
    test_class_map = {}
    
    def __init__(self):
        with open("../../Data/unique_classes.pkl","rb") as f:
            self.uc = pkl.load(f)
        self.uc = sorted(self.uc)
        self.im_class_map = {k: [] for k in self.uc}
        self.train_class_map = {k: [] for k in self.uc}
        self.test_class_map = {k: [] for k in self.uc}

        
    def create_directories_anew(self):
        try:
            shutil.rmtree("../../Data/Class wise Data")
            os.mkdir("../../Data/Class wise Data")
            os.mkdir("../../Data/Class wise Data/train")
            os.mkdir("../../Data/Class wise Data/test")
            for i in self.uc:
                os.mkdir("../../Data/Class wise Data/train/"+i)
                os.mkdir("../../Data/Class wise Data/test/"+i)
        except:
            os.mkdir("../../Data/Class wise Data")
            os.mkdir("../../Data/Class wise Data/train")
            os.mkdir("../../Data/Class wise Data/test")
            for i in self.uc:
                os.mkdir("../../Data/Class wise Data/train/"+i)
                os.mkdir("../../Data/Class wise Data/test/"+i)
    
    def create_directoy_class_map(self):
        df = pd.read_csv("../../occurrences_train.csv", low_memory=False)
        for i in range(df.shape[0]):
            im_id = df['patch_id'][i]
            im_dir = df['patch_dirname'][i]
            cls = df['class'][i]
            self.im_class_map[cls].append((im_dir,im_id,cls))
        self.im_class_map = OrderedDict(sorted(self.im_class_map.items()))
    
    def splits_size_gen(self,train_size=0.85):
        for i in self.uc:
            self.splits.append(int(train_size*len(self.im_class_map[i])))
        
    def train_test_class_map(self):
        for k in self.im_class_map.keys():
            random.shuffle(self.im_class_map[k])
            for i in range(len(self.im_class_map[k])):
                if(i<self.splits[list(self.im_class_map.keys()).index(k)]):
                    self.train_class_map[k].append((self.im_class_map[k][i],0))
                else:
                    self.test_class_map[k].append((self.im_class_map[k][i],1))
        self.train_class_map = OrderedDict(sorted(self.train_class_map.items()))
        self.test_class_map = OrderedDict(sorted(self.test_class_map.items()))
    
    def paralleized_copying(self,item):
        global i
        i+=1
        if(i%100 == 0): print(i)
        p1 = item[0][0]
        p2 = item[0][1]
        p3 = item[0][2]
        if(item[1] == 0):
            p4 = "train"
        else:
            p4 = "test"
        source = "../../patchTrain/"+str(p1)+"/patch_"+str(p2)+".tif"
        dest = "../../Data/Class wise Data/"+str(p4)+"/"+str(p3)+"/"
        shutil.copy(source,dest)
        
    def copy_data(self):
        for k in self.train_class_map.keys():
            print(k," training copy")
            i=0
            with Pool(10) as p:
                p.map(self.paralleized_copying,self.train_class_map[k])
                p.close()
                p.join()
        for k in self.test_class_map.keys():
            print(k," testing copy")
            i=0
            with Pool(10) as p:
                p.map(self.paralleized_copying,self.test_class_map[k])
                p.close()
                p.join()

In [3]:
ob = data_preprocessing()

In [4]:
ob.create_directories_anew()

In [5]:
ob.create_directoy_class_map()

In [6]:
ob.splits_size_gen(train_size=0.7)

In [7]:
ob.train_test_class_map()

In [8]:
ob.copy_data()

Cycadopsida  training copy
Equisetopsida  training copy
Ginkgoopsida  training copy
Gnetopsida  training copy
Liliopsida  training copy
100
200
100
300
100
200
100
100
400
100
300
200
100
200
100
100
200
100
500
200
400
200
300
200
300
200
300
600
200
300
500
300
400
300
400
300
700
400
300
400
600
400
500
500
400
400
500
400
800
500
700
600
500
600
600
500
900
500
500
600
800
600
600
700
700
600
1000
700
600
700
900
800
800
700
700
1100
800
700
700
800
1000
900
900
1200
800
800
800
900
1100
800
1000
1000
900
1300
900
900
1000
900
1200
900
1100
1100
1000
1400
1000
1000
1100
1000
1200
1000
1300
1200
1500
1100
1100
1100
1200
1400
1300
1100
1100
1300
1600
1200
1200
1300
1200
1500
1400
1700
1400
1200
1200
1300
1300
1600
1400
1300
1500
1500
1800
1300
1300
1400
1400
1700
1400
1500
1600
1600
1900
1400
1400
1500
1800
1500
1500
1600
1700
1500
2000
1700
1500
1600
1600
1600
1900
1800
1600
1700
1800
2100
1600
1700
1700
1700
2000
1900
1800
2200
1700
1900
1700
1800
1800
1800
2100
2000
2300
1900
1800

500
500
700
500
500
600
700
500
600
700
600
600
600
800
700
800
600
800
700
700
600
700
900
700
900
800
900
700
800
700
800
1000
800
1000
800
900
800
1000
900
900
1100
900
1000
900
900
1000
1000
1000
1200
1000
1000
Lycopodiopsida  testing copy
Magnoliopsida  testing copy
100
200
100
300
100
200
100
400
100
100
200
300
100
200
100
500
100
200
100
200
400
200
300
300
200
200
600
200
300
300
500
300
400
300
400
700
300
400
300
400
600
400
500
400
800
400
500
500
400
500
700
500
600
500
500
900
600
600
500
600
800
600
600
600
700
700
1000
600
700
700
900
700
700
700
800
800
700
1100
800
800
1000
800
800
800
800
900
900
900
1200
900
900
1100
900
900
1000
900
1000
1000
1000
1300
1000
1200
1000
1000
1100
1100
1100
1000
1100
1100
1400
1300
1100
1100
1200
1200
1200
1100
1200
1500
1200
1200
1200
1400
1300
1300
1200
1300
1300
1600
1300
1300
1500
1300
1400
1300
1400
1400
1700
1400
1400
1600
1400
1500
1400
1400
1500
1500
1500
1800
1700
1500
1600
1500
1500
1500
1600
1600
1600
1900
1800
1600
1600
160